# Import

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
import seaborn as sns
import umap.umap_ as umap

from math import ceil
from matplotlib import cm
from plotly import express as px
from scipy.cluster import hierarchy as hc
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import MinMaxScaler
from sqlalchemy import create_engine

## Funções de suporte

In [2]:
def data_summary(num_attributes):
    # Central tendency - mean, median
    tc1 = pd.DataFrame(num_attributes.apply(np.mean)).T
    tc2 = pd.DataFrame(num_attributes.apply(np.median)).T
    
    # Variability - std, coefficient of variation, min, max, range, skew, kurtosis
    d1 = pd.DataFrame(num_attributes.apply(min)).T
    d2 = pd.DataFrame(num_attributes.apply(max)).T
    d3 = pd.DataFrame(num_attributes.apply(lambda x: x.max()-x.min())).T
    d4 = pd.DataFrame(num_attributes.apply(np.std)).T
    d5 = pd.DataFrame(num_attributes.apply(lambda x: x.skew())).T
    d6 = pd.DataFrame(num_attributes.apply(lambda x: x.kurtosis())).T
    d7 = d4/tc1# cv
    
    # Concatenate
    s = pd.concat([d1, d2, d3, tc1, tc2, d4, d7, d5, d6]).T.reset_index()
    s.columns = ['columns', 'min', 'max', 'range', 'mean', 'median', 'std', 'cv', 'skew', 'kurtosis']

    return s

## Importar dados

In [3]:
path_local = '../data/raw/'
df_raw = pd.read_csv(path_local+'Ecommerce.csv', encoding = 'latin-1')

# path_s3 = 's3://insiders-dataset/'
# df_raw = pd.read_csv(path_s3+'Ecommerce.csv', encoding = 'iso-8859-1')

df_raw.drop('Unnamed: 8', axis = 1, inplace = True)

# 1 Descrição dos dados

In [4]:
df1 = df_raw.copy()

## 1.1 Renomear colunas

In [5]:
cols = ['invoice', 'stock_code', 'description', 'quantity', 'invoice_date', 'unit_price', 'customer_id', 'country']
df1.columns = cols

In [6]:
df1.columns

Index(['invoice', 'stock_code', 'description', 'quantity', 'invoice_date',
       'unit_price', 'customer_id', 'country'],
      dtype='object')

## 1.2 Dimensão dos dados

In [7]:
print('Número de linhas: {}'.format(df1.shape[0]))
print('Número de colunas: {}'.format(df1.shape[1]))

Número de linhas: 541909
Número de colunas: 8


## 1.3 Tratar os NAs

In [8]:
df1['description'].fillna('-', inplace = True)
df1.dropna(inplace = True)

In [9]:
df1.isna().sum()

invoice         0
stock_code      0
description     0
quantity        0
invoice_date    0
unit_price      0
customer_id     0
country         0
dtype: int64

In [10]:
print('Dados removidos: {:.1f}%'.format(100*(1-df1.shape[0]/df_raw.shape[0])))

Dados removidos: 24.9%


## 1.4 Tipo dos dados

In [11]:
df1['invoice_date'] = pd.to_datetime(df1['invoice_date'], format = '%d-%b-%y')
df1['customer_id'] = df1['customer_id'].astype(int)

In [12]:
df1.dtypes

invoice                 object
stock_code              object
description             object
quantity                 int64
invoice_date    datetime64[ns]
unit_price             float64
customer_id              int32
country                 object
dtype: object

## 1.5 Estatística descritiva

In [13]:
num_attributes = df1[['quantity', 'unit_price']]
cat_attributes = df1[['invoice', 'stock_code', 'description', 'customer_id', 'country']]

### 1.5.1 Atributos numéricos

In [14]:
data_summary(num_attributes)

,columns,min,max,range,mean,median,std,cv,skew,kurtosis
0,quantity,-80995.0,80995.0,161990.0,12.061303,5.00,248.693064,20.619087,0.182663,94317.563673
1,unit_price,0.0,38970.0,38970.0,3.460471,1.95,69.315077,20.030532,452.219019,246924.542988


### 1.5.2 Atributos categóricos

In [15]:
cat_attributes.apply(lambda x: x.nunique())

invoice        22190
stock_code      3684
description     3896
customer_id     4372
country           37
dtype: int64

# 2 Filtragem das variáveis

In [16]:
df2 = df1.copy()

## 2.1 Stock code

In [17]:
size_of_data = len(df2)

In [18]:
# Remover stock code M, POST, PADS, DOT, CRUK
bad_code = ['POST', 'D', 'DOT', 'M', 'S', 'AMAZONFEE', 'm', 'DCGSSBOY', 'DCGSSGIRL', 'PADS', 'B', 'CRUK']
df2 = df2[~df2['stock_code'].isin(bad_code)]

## 2.2 Quantity & price

In [19]:
# Remover quantidades negativas dos clientes não identificados
df2 = df2.drop(df2[(df2['quantity'] < 0) & (df2['customer_id'] >= 19000)].index)

In [20]:
# Remover itens com preço zero
df2 = df2[df2['unit_price'] > 0]

## 2.3 Description

In [21]:
df2.loc[df2['description'].apply(lambda x: x.isalpha()), 'description'].unique()

array(['CARRIAGE'], dtype=object)

In [22]:
# Remover description CARRIAGE
df2 = df2[df2['description'] != 'CARRIAGE']

## 2.4 Customer id

In [23]:
# Clientes cujos id possuem mais registros de devolução do que compra.
bad_customers = [12605, 12666, 12870, 12943, 13693, 
                 13829, 13958, 14119, 14213, 14627, 
                 14679, 14777, 15638, 15728, 16061, 
                 16138, 16252, 16262, 16428, 16579, 
                 16742, 16995, 17307, 17547, 17820, 
                 18141, 18256, 15823, 17548, 16546, 12346, 16446]
df2 = df2[~df2['customer_id'].isin(bad_customers)]

## 2.5 Dados filtrados

In [24]:
print('Dados removidos: {:.1f}%'.format(100*(1-df2.shape[0]/size_of_data)))
print('Número de linhas: {}'.format(df2.shape[0]))

Dados removidos: 0.5%
Número de linhas: 404734


# 3 Feature engineering

In [25]:
df3 = df2.copy()

## 3.1 Criação das variáveis

In [26]:
purchases = df3[~df3['invoice'].str.startswith('C')]
charge_back = df3[df3['invoice'].str.startswith('C')]

In [27]:
# Revenue
purchases['price'] = purchases['quantity']*purchases['unit_price']
charge_back['price'] = charge_back['quantity']*charge_back['unit_price']
df_customer = purchases[['customer_id', 'price']].groupby('customer_id').sum().reset_index().rename(columns = {'price': 'revenue'})

# # Return
# aux = charge_back[['customer_id', 'price']].groupby('customer_id').sum().reset_index().rename(columns = {'price': 'return'})
# df_customer = df_customer.merge(aux, how = 'left', on = 'customer_id')
# df_customer['return'].fillna(0, inplace = True)

# Invoices - number of purchases
aux = purchases[['customer_id', 'invoice']].groupby('customer_id').nunique().reset_index().rename(columns = {'invoice': 'invoices'})
df_customer = df_customer.merge(aux, how = 'left', on = 'customer_id')

# Invoices return
aux = charge_back[['customer_id', 'invoice']].groupby('customer_id').nunique().reset_index().rename(columns = {'invoice': 'invoices_return'})
df_customer = df_customer.merge(aux, how = 'left', on = 'customer_id')
df_customer['invoices_return'].fillna(0, inplace = True)

# Last invoice date
aux = purchases[['customer_id', 'invoice_date']].groupby('customer_id').max().reset_index().rename(columns = {'invoice_date': 'last_date'})
df_customer = df_customer.merge(aux, how = 'left', on = 'customer_id')

# Recency - days since last purchase
ref_date = df_customer['last_date'].max()
df_customer['recency'] = df_customer['last_date'].apply(lambda x: (ref_date-x).days)

# Average recency
aux = purchases[['customer_id', 'invoice_date']].groupby('customer_id').min().reset_index().rename(columns = {'invoice_date': 'first_purch'})
df_customer = df_customer.merge(aux, how = 'left', on = 'customer_id')
df_customer['avg_recency'] = df_customer.apply(lambda x: (ref_date-x['first_purch']).days/x['invoices'], axis = 1)

# Average ticket
df_purch = purchases[['customer_id', 'invoice', 'price', 'quantity']].groupby(['customer_id', 'invoice']).sum().reset_index()
df_customer['avg_ticket'] = df_purch.groupby('customer_id').mean().reset_index()['price']

# # Low purch
# df_customer['low_purch'] = df_purch.groupby('customer_id').min().reset_index()['price']

# # High purch
# df_customer['high_purch'] = df_purch.groupby('customer_id').max().reset_index()['price']

# Mean basket size
df_customer['basket_size'] = df_purch.groupby('customer_id').mean().reset_index()['quantity']

# # Country
# customers = purchases['customer_id'].unique()
# country = []
# for customer in customers:
#     aux = purchases.loc[purchases['customer_id'] == customer]
#     last_date = aux['invoice_date'].max()
#     country.append(aux[aux['invoice_date'] == last_date]['country'].iloc[0])
# df_country = pd.DataFrame({'customer_id': customers, 'country': country})
# df_customer = df_customer.merge(df_country, how = 'left', on = 'customer_id')

C:\Users\lucas\AppData\Local\Temp/ipykernel_24752/452718975.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  purchases['price'] = purchases['quantity']*purchases['unit_price']
C:\Users\lucas\AppData\Local\Temp/ipykernel_24752/452718975.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  charge_back['price'] = charge_back['quantity']*charge_back['unit_price']


In [28]:
# # Low unit price
# auxgroup = df3[['customer_id', 'unit_price']].groupby('customer_id')
# aux = auxgroup.min().reset_index().rename(columns = {'unit_price': 'low_unit_price'})
# df_customer = df_customer.merge(aux, how = 'left', on = 'customer_id')

# # High unit price
# aux = auxgroup.max().reset_index().rename(columns = {'unit_price': 'high_unit_price'})
# df_customer = df_customer.merge(aux, how = 'left', on = 'customer_id')

# # Avg unit price
# aux = auxgroup.mean().reset_index().rename(columns = {'unit_price': 'avg_unit_price'})
# df_customer = df_customer.merge(aux, how = 'left', on = 'customer_id')

# # Product variety
# aux = df3[['customer_id', 'stock_code']].groupby('customer_id').nunique().reset_index().rename(columns = {'stock_code': 'product_variety'})
# df_customer = df_customer.merge(aux, how = 'left', on = 'customer_id')

# Vintage
df_customer['vintage'] = df_customer.apply(lambda x: (ref_date-x['first_purch']).days/30, axis = 1)
df_customer['vintage'] = df_customer['vintage'].apply(ceil)

# Monthly invoices
df_customer['monthly_invoices'] = df_customer.apply(lambda x: x['invoices'] if x['vintage'] == 0 
                                                    else x['invoices']/x['vintage'], axis = 1)

# Monthly revenue
df_customer['monthly_revenue'] = df_customer.apply(lambda x: x['revenue'] if x['vintage'] == 0 
                                                    else x['revenue']/x['vintage'], axis = 1)

In [29]:
df_customer.isna().sum()

customer_id         0
revenue             0
invoices            0
invoices_return     0
last_date           0
recency             0
first_purch         0
avg_recency         0
avg_ticket          0
basket_size         0
vintage             0
monthly_invoices    0
monthly_revenue     0
dtype: int64

In [30]:
df_customer['invoices_return'] = df_customer['invoices_return'].astype(int)

In [31]:
df3 = df_customer.drop(columns = ['last_date', 'first_purch']).copy()

# 5 Preparação dos dados

In [42]:
df5 = df3.copy()

## 5.2 Rescaling

In [43]:
mm = MinMaxScaler()

# revenue
df5['revenue'] = mm.fit_transform(df5[['revenue']].values)
pickle.dump(mm, open('../src/features/revenue_scaler.pkl', 'wb'))

# invoices
df5['invoices'] = mm.fit_transform(df5[['invoices']].values)
pickle.dump(mm, open('../src/features/invoices_scaler.pkl', 'wb'))

# invoices_return
df5['invoices_return'] = mm.fit_transform(df5[['invoices_return']].values)
pickle.dump(mm, open('../src/features/invoices_return_scaler.pkl', 'wb'))

# recency
df5['recency'] = mm.fit_transform(df5[['recency']].values)
pickle.dump(mm, open('../src/features/recency_scaler.pkl', 'wb'))

# avg_recency
df5['avg_recency'] = mm.fit_transform(df5[['avg_recency']].values)
pickle.dump(mm, open('../src/features/avg_recency_scaler.pkl', 'wb'))

# avg_ticket
df5['avg_ticket'] = mm.fit_transform(df5[['avg_ticket']].values)
pickle.dump(mm, open('../src/features/avg_ticket_scaler.pkl', 'wb'))

# basket_size
df5['basket_size'] = mm.fit_transform(df5[['basket_size']].values)
pickle.dump(mm, open('../src/features/basket_size_scaler.pkl', 'wb'))

# vintage
df5['vintage'] = mm.fit_transform(df5[['vintage']].values)
pickle.dump(mm, open('../src/features/vintage_scaler.pkl', 'wb'))

# monthly_invoices
df5['monthly_invoices'] = mm.fit_transform(df5[['monthly_invoices']].values)
pickle.dump(mm, open('../src/features/monthly_invoices_scaler.pkl', 'wb'))

# monthly_revenue
df5['monthly_revenue'] = mm.fit_transform(df5[['monthly_revenue']].values)
pickle.dump(mm, open('../src/features/monthly_revenue_scaler.pkl', 'wb'))

# 6 Seleção das features

In [34]:
df5.columns

Index(['customer_id', 'revenue', 'invoices', 'invoices_return', 'recency',
       'avg_recency', 'avg_ticket', 'basket_size', 'vintage',
       'monthly_invoices', 'monthly_revenue'],
      dtype='object')

In [35]:
selected_features = ['customer_id', 'revenue', 'invoices', 'invoices_return', 
                     'recency', 'avg_recency', 'avg_ticket', 'basket_size', 
                     'vintage', 'monthly_invoices', 'monthly_revenue']
df6 = df5[selected_features].copy()

## 6.1 UMAP

In [36]:
X = df6.drop(columns = 'customer_id').copy()

In [37]:
reducer = umap.UMAP(random_state = 73)
embedding = reducer.fit_transform(X)

df_umap = pd.DataFrame(embedding)

# 8 Machine learning

In [38]:
X = df_umap.copy()

## 8.2 Hierarchical Clustering

In [39]:
n_clusters = 10

# Model
hc_model = hc.linkage(X, 'ward')

# Training & redict
labels = hc.fcluster(hc_model, n_clusters, criterion = 'maxclust')

# Validation
print('SS: {}'.format(silhouette_score(X, labels, metric='euclidean')))

SS: 0.5414782166481018


# 9 Análise do cluster

## 9.2 Recuperar dados originais

In [40]:
df9 = df3.copy()
df9['cluster'] = labels

# 11 Deploy

In [44]:
df11 = df9.copy()

## 11.1 Inserir dados no SQLite

In [45]:
# Conexão com o banco de dados
engine = create_engine('sqlite:///../src/data/insider.sqlite', echo = False)

# Criar tabela
query_create_table_insiders = '''
CREATE TABLE insider(
    customer_id         INTEGER,
    revenue             REAL,
    invoices            INTEGER,
    invoices_return     INTEGER,
    recency             INTEGER,
    avg_recency         REAL,
    avg_ticket          REAL,
    basket_size         REAL,
    vintage             INTEGER,
    monthly_invoices    REAL,
    monthly_revenue     REAL,
    cluster             INTEGER
)
'''
conn = engine.connect()
conn.execute(query_create_table_insiders)

# Inserir dados
df11.to_sql('insider', con = conn, if_exists = 'append', index = False)

# Fechar conexão
conn.close()

## 11.2 API tester

In [1]:
import json
import pandas as pd
import requests

In [2]:
df = pd.read_csv('../data/raw/Ecommerce.csv', encoding = 'latin-1')
data = json.dumps(df.to_dict(orient = 'records'))

'Connection aborted.', error(10054, 'An existing connection was forcibly closed by the remote Host')

In [9]:
url = 'http://127.0.0.1:5000/cluster'
# url = 'https://rossmann-store-sales.herokuapp.com/predict'
header = {'Content-type': 'application/json'}

# API call
r = requests.post(url, data = data, headers = header)
print('Status code {}'.format(r.status_code))

Status code 200


In [10]:
d1 = pd.DataFrame(r.json(), columns = r.json()[0].keys())

In [11]:
d1.head()

,customer_id,revenue,invoices,invoices_return,recency,avg_recency,avg_ticket,basket_size,vintage,monthly_invoices,monthly_revenue,cluster
0,12347,4310.00,7,0,2,52.428571,615.714286,351.142857,13,0.538462,331.538462,4
1,12348,1437.24,4,0,75,89.500000,359.310000,583.000000,12,0.333333,119.770000,4
2,12349,1457.55,1,0,18,18.000000,1457.550000,630.000000,1,1.000000,1457.550000,5
3,12350,294.40,1,0,310,310.000000,294.400000,196.000000,11,0.090909,26.763636,8
4,12352,1385.74,7,1,36,42.285714,197.962857,75.142857,10,0.700000,138.574000,2
